# Исследование способов определения контуров документов на тестовых фото и их поворот в горизонтальную или вертикальную плоскость
- Автор: Кирилл Киселев
- Дата начала: 21.06.2022
- Описание исследования: имеется 6 тестовых фото документов. Необходимо исследовать качество определения контуров этих документов и их поворот в горизаонтальную или вертикальную плоскость.

## 1. Имеющиеся изображения

### Исходный список
1. Расположен в папке '/home/kirillk/datasets/OCR/ocr_test_cases_6/'
    - 'IMG_20220621_102508.jpg',
    - 'IMG_20220621_102511.jpg',
    - 'IMG_20220621_102530.jpg',
    - 'IMG_20220621_102559.jpg',
    - 'IMG_20220621_102627.jpg',
    - 'IMG_20220621_102645.jpg'

## 2. Загрузка библиотек

In [1]:
import cv2
import imutils
import numpy as np
import matplotlib as plt
import os
import math
from PIL import Image
from pyzbar import pyzbar

## 3. Глобальные переменные

In [3]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks


In [2]:
FILE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases_6/'

## 4. Необходимые функции

In [3]:
def detect_shape(contour):
    """Returns the shape (e.g. 'triangle', 'square') from the contour"""

    detected_shape = '-----'

    # Calculate perimeter of the contour:
    perimeter = cv2.arcLength(contour, True)

    # Get a contour approximation:
    contour_approx = cv2.approxPolyDP(contour, 0.03 * perimeter, True)

    # Check if the number of vertices is 3. In this case, the contour is a triangle
    if len(contour_approx) == 3:
        detected_shape = 'triangle'

    # Check if the number of vertices is 4. In this case, the contour is a square/rectangle
    elif len(contour_approx) == 4:

        # We calculate the aspect ration from the bounding rect:
        x, y, width, height = cv2.boundingRect(contour_approx)
        aspect_ratio = float(width) / height

        # A square has an aspect ratio close to 1 (comparison chaining is used):
        if 0.90 < aspect_ratio < 1.10:
            detected_shape = "square"
        else:
            detected_shape = "rectangle"

    # Check if the number of vertices is 5. In this case, the contour is a pentagon
    elif len(contour_approx) == 5:
        detected_shape = "pentagon"

    # Check if the number of vertices is 6. In this case, the contour is a hexagon
    elif len(contour_approx) == 6:
        detected_shape = "hexagon"

    # The shape as more than 6 vertices. In this example, we assume that is a circle
    else:
        detected_shape = "circle"

    # return the name of the shape and the found vertices
    return detected_shape, contour_approx

## 5. Данные

In [4]:
# список фото
photo_list = sorted([photo for photo in os.listdir(FILE_PATH) if photo.endswith('.jpg')])
photo_list

['IMG_20220621_102508.jpg',
 'IMG_20220621_102511.jpg',
 'IMG_20220621_102530.jpg',
 'IMG_20220621_102559.jpg',
 'IMG_20220621_102627.jpg',
 'IMG_20220621_102645.jpg']

In [6]:
# посмотрим на фото

for photo in photo_list:
    image = cv2.imread(FILE_PATH + photo)
    image = cv2.resize(image, (0, 0), fx=0.4, fy=0.4)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 6. Определение контуров

In [8]:
im = cv2.imread(FILE_PATH + photo_list[0])
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])
# print(vertices)
img = cv2.polylines(im, [vertices], True, (0,0,255), 2)
img = cv2.resize(img, (0, 0), fx=0.1, fy=0.1)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
# посмотрим на все фото
for photo in photo_list:
    im = cv2.imread(FILE_PATH + photo)
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 100, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num

    shape, vertices = detect_shape(contours[num_of_contour])
    # print(vertices)
    img = cv2.polylines(im, [vertices], True, (0,0,255), 6)
    img = cv2.resize(img, (0, 0), fx=0.1, fy=0.1)

    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Вывод
Как видно из результатов работы скрипта - удалось определить контуры фото на всех документах.

Переходим к проверке поворота.

## 7. Поворот фото

### Для примера обработаем первое фото из списка.

In [17]:
im = cv2.imread(FILE_PATH + photo_list[0])
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 100, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])
print(shape, vertices)
img = cv2.polylines(im, [vertices], True, (0, 0, 255), 8)
img = cv2.resize(img, (0, 0), fx=0.12, fy=0.12)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

rectangle [[[8693  530]]

 [[ 503  781]]

 [[ 568 6201]]

 [[8552 6656]]]


In [21]:
im = cv2.imread(FILE_PATH + photo_list[0])
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(im_gray, 100, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])

x1 = (vertices[1][0][0])
y1 = (vertices[1][0][1])

x2 = (vertices[2][0][0])
y2 = (vertices[2][0][1])

xM = (vertices[1][0][0])
yM = (vertices[2][0][1])

catet_M_2 = abs(x2 - xM)
catet_1_M = abs(yM - y1)

tg_a = catet_1_M / catet_M_2
angle = round(math.degrees(math.atan(tg_a)))

# здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
img = imutils.rotate_bound(im_gray, -angle)

# снова ищем контуры документа
_, thresh = cv2.threshold(img, 100, 255, 0)  
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

_, vertices_2 = detect_shape(contours[num_of_contour])

x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])
print(x_1, x_2)
print(y_1, y_2)

# img = cv2.polylines(img, [vertices_2], True, (0, 0, 255), 8)
# print(vertices_2)

im = imutils.rotate_bound(im, -angle)

im = im[y_1:y_2, x_1:x_2]

im = cv2.resize(im, (0, 0), fx=0.12, fy=0.12)


# img = img[y_1:y_2, x_1:x_2]
# img = cv2.resize(img, (0, 0), fx=0.15, fy=0.15)

cv2.imshow("Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

681 6803
564 8787


### А теперь все остальные

In [23]:
def find_contour(gray_image, threshold=100):
    _, thresh = cv2.threshold(gray_image, threshold, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num
    return contours[num_of_contour]    

In [12]:
counter = 0

for photo in photo_list:
    im = cv2.imread(FILE_PATH + photo)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    doc_contour = find_contour(im_gray)
    shape, vertices = detect_shape(doc_contour)

    assert shape=='rectangle'

    x1, y1 = vertices[1][0][0], vertices[1][0][1]
    x2, y2 = vertices[2][0][0], vertices[2][0][1]
    xM, yM = vertices[1][0][0], vertices[2][0][1]

    # catet_M_2 = abs(x2 - xM)
    # catet_1_M = abs(yM - y1)

    catet_M_2 = (x2 - xM)
    catet_1_M = (yM - y1)

    tg_a = catet_1_M / catet_M_2
    angle = round(math.degrees(math.atan(tg_a)))

    # здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
    img = imutils.rotate_bound(im_gray, -angle)

    # снова ищем контуры документа
    contour_for_crop = find_contour(img)
    _, vertices_2 = detect_shape(contour_for_crop)

    x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
    y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])

    im = imutils.rotate_bound(im, -angle)
    im = im[y_1:y_2, x_1:x_2]
    cv2.imwrite(f'/home/kirillk/datasets/OCR/ocr_test_cases_6/old_test/{counter}.jpg', im)
    counter += 1
#     im = cv2.resize(im, (0, 0), fx=0.12, fy=0.12)

#     cv2.imshow("Image", im)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()